In [28]:
#!/usr/bin/env python3

import logging
from pathlib import Path

from Pegasus.api import *

logging.basicConfig(level=logging.DEBUG)

#Properties
props = Properties()
props["pegasus.data.configuration"] = "condorio"
props["dagman.retry"] = "2"
props.write()


#Replica Catalog
input_file = File("input.txt")
out_file = File("backup.txt")

rc = ReplicaCatalog()\
        .add_replica("local", "input.txt", Path(".").resolve() / "input.txt")\
        .write()

#Transformations
increment = Transformation(
        "increment.py",
        site="local",
        pfn=str(Path(".").resolve() / "increment.py"),
        is_stageable=True
    )

tc = TransformationCatalog()\
    .add_transformations(increment)\
    .write()

wf = Workflow("Increment", infer_dependencies=True)
job_inc =  Job(increment)\
        .add_args(input_file, out_file)\
        .add_inputs(input_file)\
        .set_stdout(out_file)

wf.add_jobs(job_inc)

In [29]:
try:
     wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)

Plan:
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.07.12 06:12:43.565 UTC:    
2020.07.12 06:12:43.570 UTC:   ----------------------------------------------------------------------- 
2020.07.12 06:12:43.576 UTC:   File for submitting this DAG to HTCondor           : Increment-0.dag.condor.sub 
2020.07.12 06:12:43.581 UTC:   Log of DAGMan debugging messages                 : Increment-0.dag.dagman.out 
2020.07.12 06:12:43.587 UTC:   Log of HTCondor library output                     : I

[##################################################] 100.0% ..Success (Completed: 9, Queued: 0, Running: 0, Failed: 0)


Analyzer:
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

 Submit Directory   : /home/scitech/shared-data/Workflow/Workflow2/scitech/pegasus/Increment/run0003
 Total jobs         :      9 (100.00%)
 # jobs succeeded   :      9 (100.00%)
 # jobs failed      :      0 (0.00%)
 # jobs held        :      0 (0.00%)
 # jobs unsubmitted :      0 (0.00%)

 

Statistics:
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workflows.
#     * Succeeded - total count of succeeded tasks/jobs/sub workflows.
#     * Failed - total c